Zero-shot learning (ZSL) is a machine learning paradigm where a model is trained to recognize objects or perform tasks that it has never seen before during training. Instead of relying solely on labeled examples for every possible category, ZSL leverages auxiliary information (such as semantic attributes, descriptions, or relationships) to make predictions about unseen classes.

# Code for Zero-Shot Learning

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cdist
import fasttext

# Load a pre-trained ResNet50 model
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to extract visual features
def extract_features(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    features = model.predict(image)
    return features

# Load FastText word vectors
fasttext_model = fasttext.load_model('cc.en.300.bin')
# from gensim.models.keyedvectors import KeyedVectors
# import gensim.downloader as api
# fast_text_vectors = api.load("fasttext-wiki-news-subwords-300")
# fast_text_vectors.save('fstwk_1.d2v')
# fast_text_vectors = KeyedVectors.load("fstwk_1.d2v")

# Example seen and unseen classes
seen_classes = ['cat', 'dog', 'horse']
unseen_classes = ['lion', 'tiger', 'elephant']

# Get word vectors for classes
def get_class_vectors(classes):
    return np.array([fasttext_model.get_word_vector(cls) for cls in classes])

# Normalize the word vectors
seen_vectors = normalize(get_class_vectors(seen_classes))
unseen_vectors = normalize(get_class_vectors(unseen_classes))

# Function to perform zero-shot classification
def zero_shot_classify(image_path):
    features = extract_features(image_path)
    features = normalize(features)
    distances = cdist(features, unseen_vectors, metric='cosine')
    return unseen_classes[np.argmin(distances)]

# Example usage
image_path = 'path_to_image_of_an_unseen_animal.jpg'
predicted_class = zero_shot_classify(image_path)
print(f'Predicted class: {predicted_class}')
